# Facial Expression Recognition

### Import Required Libraries

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, BatchNormalization,
    Flatten, Dense, Dropout
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
import kagglehub

dataset_path = kagglehub.dataset_download(
    "apollo2506/facial-recognition-dataset"
)

print("Dataset downloaded at:", dataset_path)

train_dir = os.path.join(dataset_path, "Training", "Training")
test_dir  = os.path.join(dataset_path, "Testing", "Testing")

print("Training Directory:", train_dir)
print("Testing Directory :", test_dir)

assert os.path.exists(train_dir), "Training folder not found!"
assert os.path.exists(test_dir), "Testing folder not found!"


### Exploratory Data Analysis

In [ ]:
def count_images(folder_path):
    counts = {}
    for emotion in os.listdir(folder_path):
        emotion_path = os.path.join(folder_path, emotion)
        if os.path.isdir(emotion_path):
            counts[emotion] = len(os.listdir(emotion_path))
    return counts

train_counts = count_images(train_dir)
test_counts  = count_images(test_dir)

print("Training Samples:", train_counts)
print("Testing Samples :", test_counts)

#Visualize Class Distribution

plt.figure(figsize=(8,4))
sns.barplot(
    x=list(train_counts.keys()),
    y=list(train_counts.values())
)
plt.title("Training Data Distribution")
plt.ylabel("Number of Images")
plt.xticks(rotation=45)
plt.show()

#Visualize Sample Images
from tensorflow.keras.preprocessing.image import load_img

plt.figure(figsize=(10,6))

for i, emotion in enumerate(list(train_counts.keys())[:6]):
    img_path = os.path.join(train_dir, emotion, os.listdir(os.path.join(train_dir, emotion))[0])
    img = load_img(img_path, color_mode='grayscale', target_size=(48,48))
    
    plt.subplot(2,3,i+1)
    plt.imshow(img, cmap='gray')
    plt.title(emotion)
    plt.axis('off')

plt.suptitle("Sample Images from Dataset")
plt.show()


### Data Preprocessing & Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)


### Create Data Generators

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='sparse',
    shuffle=True
)

val_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='sparse',
    shuffle=False
)

emotions = list(train_generator.class_indices.keys())
print("Detected Emotion Classes:", emotions)



### CNN Model

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3,3), activation='relu', padding='same',
                 input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.4))

model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(len(emotions), activation='softmax'))

model.summary()


model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-5,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)


history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=80,
    callbacks=[reduce_lr, early_stop],
    verbose=1
)


### Visualize Training Performance

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Loss')
plt.legend()
plt.grid(True)

plt.show()

### Model Evaluation

In [ ]:
val_generator.reset()
preds = model.predict(val_generator)
pred_classes = np.argmax(preds, axis=1)
true_classes = val_generator.classes

cm = confusion_matrix(true_classes, pred_classes)

plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=emotions,
            yticklabels=emotions)
plt.title("Confusion Matrix")
plt.show()

print("\nClassification Report:")
print(classification_report(true_classes, pred_classes,
                            target_names=emotions))

### Save the model

In [ ]:

model.save("emotion_model_v3.h5")
print("Model saved as emotion_model_v3.h5")